# Webtoon Comics Prediction

In [ ]:
# https://www.kaggle.com/code/bayunova/webtoon-comics

In [11]:
#import library
import numpy as np
import pandas as pd
import seaborn as sns
import nltk
nltk.download('stopwords')
from textblob import TextBlob
from wordcloud import WordCloud
import re
import string
string.punctuation
import matplotlib.pyplot as plt
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kaitnam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Data Extraction

In [12]:
comic = pd.read_csv("E:/Kaggle Practice/Webtoon Comics/Webtoon Dataset.csv")
comic.head()

,id,Name,Writer,Likes,Genre,Rating,Subscribers,Summary,Update,Reading Link
0,0,Let's Play,Leeanne M. Krecic (Mongie),30.6M,Romance,9.62,4.2M,"She's young, single and about to achieve her d...",UP EVERY TUESDAY,https://www.webtoons.com/en/romance/letsplay/l...
1,1,True Beauty,Yaongyi,39.9M,Romance,9.60,6.4M,"After binge-watching beauty videos online, a s...",UP EVERY WEDNESDAY,https://www.webtoons.com/en/romance/truebeauty...
2,2,Midnight Poppy Land,Lilydusk,10.4M,Romance,9.81,2.1M,After making a grisly discovery in the country...,UP EVERY SATURDAY,https://www.webtoons.com/en/romance/midnight-p...
3,3,Age Matters,Enjelicious,25.9M,Romance,9.79,3.5M,She's a hopeless romantic who's turning 30's ...,UP EVERY WEDNESDAY,https://www.webtoons.com/en/romance/age-matter...
4,4,Unholy Blood,Lina Im / Jeonghyeon Kim,9.9M,Supernatural,9.85,1.5M,When vampires destroy her chance to have the n...,UP EVERY THURSDAY,https://www.webtoons.com/en/supernatural/unhol...


In [3]:
comic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            569 non-null    int64  
 1   Name          569 non-null    object 
 2   Writer        568 non-null    object 
 3   Likes         569 non-null    object 
 4   Genre         569 non-null    object 
 5   Rating        569 non-null    float64
 6   Subscribers   569 non-null    object 
 7   Summary       569 non-null    object 
 8   Update        569 non-null    object 
 9   Reading Link  569 non-null    object 
dtypes: float64(1), int64(1), object(8)
memory usage: 44.6+ KB


In [4]:
comic.describe()

,id,Rating
count,569.000000,569.000000
mean,284.000000,9.418998
std,164.400426,0.557571
min,0.000000,5.410000
25%,142.000000,9.310000
50%,284.000000,9.580000
75%,426.000000,9.730000
max,568.000000,9.930000


In [5]:
comic.isnull().sum()

id              0
Name            0
Writer          1
Likes           0
Genre           0
Rating          0
Subscribers     0
Summary         0
Update          0
Reading Link    0
dtype: int64

In [6]:
comic.shape

(569, 10)

## Text Processing

In [13]:
# selection data
df1 = comic[['Rating', 'Genre', 'Summary']]
df1.head()

,Rating,Genre,Summary
0,9.62,Romance,"She's young, single and about to achieve her d..."
1,9.60,Romance,"After binge-watching beauty videos online, a s..."
2,9.81,Romance,After making a grisly discovery in the country...
3,9.79,Romance,She's a hopeless romantic who's turning 30's ...
4,9.85,Supernatural,When vampires destroy her chance to have the n...


In [14]:
def remove_punctuation(text):
    punctuationfree = "".join([i for i in text if i not in string.punctuation])
    #"".join(리스트)를 이용하면 매개변수로 들어온 ['a', 'b', 'c'] 이런 식의 리스트를 'abc'의 문자열로 합쳐서 반환해주는 함수인 것입니다.
    # string.punctuation을 넣으면 문자열 양쪽의 모든 구두점을 간단하게 삭제할 수 있습니다.
    # 예들들어 ', python.'.strip(string.punctuation) 이렇게 코드를 치면 구두점이 없어진 ' python' 이렇게 출력된다
    return punctuationfree
df1["Clean_Summary"] = df1["Summary"].apply(lambda x:remove_punctuation(x))
df1.head()
# 결과를 보면 Summary의 text의 구두점들이 Clean_Summary에서는 제거된거를 확인 할 수 있다.

<ipython-input-14-545a1c73b3c8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["Clean_Summary"] = df1["Summary"].apply(lambda x:remove_punctuation(x))


,Rating,Genre,Summary,Clean_Summary
0,9.62,Romance,"She's young, single and about to achieve her d...",Shes young single and about to achieve her dre...
1,9.60,Romance,"After binge-watching beauty videos online, a s...",After bingewatching beauty videos online a shy...
2,9.81,Romance,After making a grisly discovery in the country...,After making a grisly discovery in the country...
3,9.79,Romance,She's a hopeless romantic who's turning 30's ...,Shes a hopeless romantic whos turning 30s and...
4,9.85,Supernatural,When vampires destroy her chance to have the n...,When vampires destroy her chance to have the n...


In [ ]:
# lambda 설명 참조
# https://wikidocs.net/46758

In [15]:
# Setting lower case
df1["Summary_Lower"] = df1["Clean_Summary"].apply(lambda x: x.lower())
df1.head()

<ipython-input-15-3418a9e96f54>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["Summary_Lower"] = df1["Clean_Summary"].apply(lambda x: x.lower())


,Rating,Genre,Summary,Clean_Summary,Summary_Lower
0,9.62,Romance,"She's young, single and about to achieve her d...",Shes young single and about to achieve her dre...,shes young single and about to achieve her dre...
1,9.60,Romance,"After binge-watching beauty videos online, a s...",After bingewatching beauty videos online a shy...,after bingewatching beauty videos online a shy...
2,9.81,Romance,After making a grisly discovery in the country...,After making a grisly discovery in the country...,after making a grisly discovery in the country...
3,9.79,Romance,She's a hopeless romantic who's turning 30's ...,Shes a hopeless romantic whos turning 30s and...,shes a hopeless romantic whos turning 30s and...
4,9.85,Supernatural,When vampires destroy her chance to have the n...,When vampires destroy her chance to have the n...,when vampires destroy her chance to have the n...


In [16]:
# Defining function for tokenization
def tokenization(text):
    tokens = re.split("W+", text)
    return tokens

# Applying function to the column
df1["Summary_Tokenied"] = df1["Summary_Lower"].apply(lambda x : tokenization(x))
df1.head()

<ipython-input-16-b6d919f89370>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["Summary_Tokenied"] = df1["Summary_Lower"].apply(lambda x : tokenization(x))


,Rating,Genre,Summary,Clean_Summary,Summary_Lower,Summary_Tokenied
0,9.62,Romance,"She's young, single and about to achieve her d...",Shes young single and about to achieve her dre...,shes young single and about to achieve her dre...,[shes young single and about to achieve her dr...
1,9.60,Romance,"After binge-watching beauty videos online, a s...",After bingewatching beauty videos online a shy...,after bingewatching beauty videos online a shy...,[after bingewatching beauty videos online a sh...
2,9.81,Romance,After making a grisly discovery in the country...,After making a grisly discovery in the country...,after making a grisly discovery in the country...,[after making a grisly discovery in the countr...
3,9.79,Romance,She's a hopeless romantic who's turning 30's ...,Shes a hopeless romantic whos turning 30s and...,shes a hopeless romantic whos turning 30s and...,[shes a hopeless romantic whos turning 30s an...
4,9.85,Supernatural,When vampires destroy her chance to have the n...,When vampires destroy her chance to have the n...,when vampires destroy her chance to have the n...,[when vampires destroy her chance to have the ...


In [17]:
# Stop words present in the library
stopwords = nltk.corpus.stopwords.words('english')
stopwords[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [19]:
# Defining the function to remove stopwords from tokenized text 
def remove_stopwords(text):
    output = [i for i in text if i not in stopwords]
    return output

# Applyinh the function
df1["No_Stopwords"] = df1["Summary_Tokenied"].apply(lambda x : remove_stopwords(x))
df1.head()

<ipython-input-19-827f6a1fbf3c>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["No_Stopwords"] = df1["Summary_Tokenied"].apply(lambda x : remove_stopwords(x))


,Rating,Genre,Summary,Clean_Summary,Summary_Lower,Summary_Tokenied,No_Stopwords
0,9.62,Romance,"She's young, single and about to achieve her d...",Shes young single and about to achieve her dre...,shes young single and about to achieve her dre...,[shes young single and about to achieve her dr...,[shes young single and about to achieve her dr...
1,9.60,Romance,"After binge-watching beauty videos online, a s...",After bingewatching beauty videos online a shy...,after bingewatching beauty videos online a shy...,[after bingewatching beauty videos online a sh...,[after bingewatching beauty videos online a sh...
2,9.81,Romance,After making a grisly discovery in the country...,After making a grisly discovery in the country...,after making a grisly discovery in the country...,[after making a grisly discovery in the countr...,[after making a grisly discovery in the countr...
3,9.79,Romance,She's a hopeless romantic who's turning 30's ...,Shes a hopeless romantic whos turning 30s and...,shes a hopeless romantic whos turning 30s and...,[shes a hopeless romantic whos turning 30s an...,[shes a hopeless romantic whos turning 30s an...
4,9.85,Supernatural,When vampires destroy her chance to have the n...,When vampires destroy her chance to have the n...,when vampires destroy her chance to have the n...,[when vampires destroy her chance to have the ...,[when vampires destroy her chance to have the ...


In [20]:
from nltk.corpus import stopwords  
stopwords.words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [21]:
# Defining the object for stemming 
porter_stemmer = PorterStemmer()

# Defining a function for stemming 
def stemming(text):
    stem_text = [porter_stemmer.stem(word) for word in text]
    return stem_text

df1['Summary_Stemmed'] = df1["No_Stopwords"].apply(lambda x : stemming(x))
df1.head()

<ipython-input-21-f92c29cedf99>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Summary_Stemmed'] = df1["No_Stopwords"].apply(lambda x : stemming(x))


,Rating,Genre,Summary,Clean_Summary,Summary_Lower,Summary_Tokenied,No_Stopwords,Summary_Stemmed
0,9.62,Romance,"She's young, single and about to achieve her d...",Shes young single and about to achieve her dre...,shes young single and about to achieve her dre...,[shes young single and about to achieve her dr...,[shes young single and about to achieve her dr...,[shes young single and about to achieve her dr...
1,9.60,Romance,"After binge-watching beauty videos online, a s...",After bingewatching beauty videos online a shy...,after bingewatching beauty videos online a shy...,[after bingewatching beauty videos online a sh...,[after bingewatching beauty videos online a sh...,[after bingewatching beauty videos online a sh...
2,9.81,Romance,After making a grisly discovery in the country...,After making a grisly discovery in the country...,after making a grisly discovery in the country...,[after making a grisly discovery in the countr...,[after making a grisly discovery in the countr...,[after making a grisly discovery in the countr...
3,9.79,Romance,She's a hopeless romantic who's turning 30's ...,Shes a hopeless romantic whos turning 30s and...,shes a hopeless romantic whos turning 30s and...,[shes a hopeless romantic whos turning 30s an...,[shes a hopeless romantic whos turning 30s an...,[shes a hopeless romantic whos turning 30s an...
4,9.85,Supernatural,When vampires destroy her chance to have the n...,When vampires destroy her chance to have the n...,when vampires destroy her chance to have the n...,[when vampires destroy her chance to have the ...,[when vampires destroy her chance to have the ...,[when vampires destroy her chance to have the ...


In [ ]:
# Porter_Stemmer 정보
# https://wikidocs.net/21707

## Sentiment Analysis

In [25]:
# Selection data
df2 = df1[["Rating", "Genre", "Clean_Summary"]]
# 대괄호가 2개임
df2.head()

,Rating,Genre,Clean_Summary
0,9.62,Romance,Shes young single and about to achieve her dre...
1,9.60,Romance,After bingewatching beauty videos online a shy...
2,9.81,Romance,After making a grisly discovery in the country...
3,9.79,Romance,Shes a hopeless romantic whos turning 30s and...
4,9.85,Supernatural,When vampires destroy her chance to have the n...


In [ ]:
# Create function to get subjectivity
def getSubjectivity(text):
    